## 問題

https://twitter.com/e869120/status/1385001057512693762

## 解説

https://twitter.com/e869120/status/1385363292739104775

In [1]:
#r "nuget: FSharpPlus"

Installed Packages FSharpPlus, 1.3.2

In [2]:
open System
open System.Numerics
open System.Collections.Generic
open FSharpPlus

In [42]:
/// 強連結成分分解
let scc edges =
  let rec dfs (nodeDict: IDictionary<int, int list>) (nodeSet: HashSet<int>) result i =
    nodeSet.Add(i) |> ignore
    let result = 
      let nodes = nodeDict.TryGetValue(i) |> Option.ofPair |> Option.defaultValue []
      (result, nodes) ||> List.fold (fun result n ->
        if nodeSet.Contains(n) then result else dfs nodeDict nodeSet result n)
    i::result
  let nodes = edges |> List.collect (fun (s, e) -> [s; e]) |> Seq.distinct |> Seq.toList
  let getNodeDict f1 f2 = edges |> Seq.groupBy f1 |> Seq.map (fun (k, vs) -> k, vs |> Seq.map f2 |> Seq.toList) |> dict
  let startNodeDict, endNodeDict = getNodeDict fst snd, getNodeDict snd fst
  let sortedNodes =
    (([], HashSet()), nodes) ||> List.fold (fun (result, nodeSet) n ->
      let result =  if nodeSet.Contains(n) then result else dfs startNodeDict nodeSet result n
      result, nodeSet)
    |> fst
  (([], HashSet()), sortedNodes) ||> List.fold (fun (result, nodeSet) n ->
    let result = if nodeSet.Contains(n) then result else (dfs endNodeDict nodeSet [] n)::result
    result, nodeSet)
  |> fst

In [46]:
let solve Ms =
  // 強連結している頂点を2つ組み合わせた場合の数の合計
  Ms |> scc |> List.sumBy (fun xs -> let len = length xs in len * (len - 1) / 2)


In [47]:
solve [
  1, 2
  2, 1
  2, 3
  4, 3
  4, 1
  1, 4
  2, 3
]

3

In [48]:
solve [
  1, 2
]

0